In [1]:
#distance square
def dist_sq(a, b):
    return np.sum((a-b)**2)
#distance square for every point to the centroid
def point_sq(data,index):
    Dist=list()
    for i in range(data.shape[0]):
        Dist.append(dist_sq(data.ix[index],data.ix[i]))
    return Dist
def dist_prob(Dist):
    return (Dist/np.sum(np.array(Dist)))

In [2]:
def kmeansplusplus(data, k, d):
    centroid=np.zeros([k,d])
    index=np.random.choice(data.shape[0],1)
    centroid[0]=data.ix[index]
    for number in range(k-1):
        distance=point_sq(data,index)
        prob=dist_prob(distance).tolist()
        #prob[index]=0
        index=np.random.choice(data.shape[0],1,prob)
        centroid[number+1]=data.ix[index]
    return centroid


In [76]:
def kmeans(dataSet, initial):
    centroids=initial
    # Initialize book keeping vars.
    iterations = 0
    oldCentroids = np.zeros([initial.shape[0],initial.shape[1]])
    
    # Run the main k-means algorithm
    while not shouldStop(oldCentroids, centroids, iterations):
        # Save old centroids for convergence test. Book keeping.
        oldCentroids = centroids
        iterations += 1
        
        # Assign labels to each datapoint based on centroids
        l= getLabels(dataSet, centroids)
        
        # Assign centroids based on datapoint labels
        centroids = getCentroids(dataSet, l)
        
    # We can get the labels too by calling getLabels(dataSet, centroids)
    return centroids, np.array(l)


In [73]:
# Function: Should Stop
# -------------
# Returns True or False if k-means is done. K-means terminates either
# because it has run a maximum number of iterations OR the centroids
# stop changing.
def shouldStop(oldCentroids, centroids, iterations):
    if iterations > 50: return True
    return oldCentroids.all == centroids.all


In [71]:
# Function: Get Labels
# -------------
# Returns a label for each piece of data in the dataset. 
def getLabels(dataSet, centroids):
    # For each element in the dataset, chose the closest centroid. 
    # Make that centroid the element's label.
    l=[]
    for i in range(dataSet.shape[0]):
        dist=np.zeros(centroids.shape[0])
        for j in range(centroids.shape[0]):
            dist[j]=dist_sq(dataSet.ix[i],centroids[j])
        l.append(np.argmin(dist))
    return l
            


In [60]:
# Function: Get Centroids
# -------------
# Returns k random centroids, each of dimension n.
def getCentroids(dataSet, labels):
    # Each centroid is the geometric mean of the points that
    # have that centroid's label. Important: If a centroid is empty (no points have
    # that centroid's label) you should randomly re-initialize it.
    data_new = dataSet.copy()
    data_new['Labels'] = labels
    data_new = data_new.groupby(['Labels']).mean().iloc[:,:2]
    return np.array(data_new)
    

In [ ]:
#test 
data=DataFrame([np.random.random(100),np.random.random(100)]).transpose()
initial= kmeansplusplus(data,8,2)

In [79]:
kmeans(data, initial)

(array([[ 0.80265401,  0.26396164],
        [ 0.30738329,  0.93670259],
        [ 0.08570629,  0.32116847],
        [ 0.4947732 ,  0.28917512],
        [ 0.26616442,  0.58670997],
        [ 0.72585359,  0.8118506 ],
        [ 0.05606172,  0.75503595],
        [ 0.22941675,  0.10854025]]),
 array([3, 0, 4, 4, 0, 3, 5, 5, 2, 2, 2, 4, 3, 0, 5, 7, 5, 0, 3, 5, 5, 5, 2,
        0, 6, 1, 3, 7, 3, 5, 1, 3, 4, 4, 3, 5, 6, 5, 1, 0, 5, 0, 5, 7, 7, 3,
        5, 0, 7, 4, 3, 0, 6, 5, 2, 1, 4, 5, 5, 3, 4, 7, 5, 5, 5, 4, 5, 0, 2,
        0, 5, 4, 7, 1, 3, 5, 7, 3, 4, 3, 4, 1, 6, 0, 0, 1, 1, 1, 2, 5, 5, 4,
        0, 5, 3, 7, 0, 3, 5, 4]))